<a href="https://colab.research.google.com/github/a2r4vind/Machine-Learning-Models/blob/main/Implementing_NeuralNetwork_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
class NaiveDense:
  def __init__(self, input_size, output_size, activation): # __init__ makes a constructor to create object of class
    self.activation = activation

    #creating a matrix, W, of shape (input_size, output_size), initialized with random values
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval= 1e-1)
    self.W = tf.Variable(w_initial_value)

    # creating a vector, b, of shape (output_size), initialized with zeros
    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)

  #Apply the forward pass.
  def __call__(self, inputs): # __call__ makes the object of class to behave as function and object can be called as a function
      return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self): # method for retrieving layer's weights
      return [self.W, self.b]


In [3]:
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer(x)
    return x

  @property
  def weights(self):
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [4]:
# Creating a model
model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [5]:
import math

In [6]:
class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)
  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

In [7]:
# Training step
def one_training_step(model, images_batch, labels_batch):
  with tf.GradientTape() as tape:
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
        labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)
  # compute the gradient of the loss with regard to the weights. The output gradients
  # is a list where each entry corresponds to a weight from model.weights list.
  gradients = tape.gradient(average_loss, model.weights)
  # Update the weights using the gradients
  update_weights(gradients, model.weights)
  return average_loss

In [10]:
learning_rate = 1e-1

def update_weights(gradients, weights):
  for g, w in zip(gradients, weights):
    w.assign_sub(g * learning_rate)

# # the above logic is made more concise using tensorflow.keras module
# from tensorflow.keras import optimizers

# optimizer = optimizers.SGD(learning_rate=1e-3)

# def update_weights(gradients, weights):
#   optimizer.apply_gradients(zip(gradients, weights))

In [18]:
# Full training loop

def fit(model, images, labels, epochs, batch_size=128):
  for epoch_counter in range(epochs):
    print(f"Epoch {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches):
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"loss at batch {batch_counter}: {loss:.2f}")

In [19]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 1
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 2
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 3
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 4
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 5
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 6
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
loss at batch 300: 14.23
loss at batch 400: 14.23
Epoch 7
loss at batch 0: 13.85
loss at batch 100: 14.86
loss at batch 200: 14.86
lo

In [20]:
import numpy as np
# Evaluating the model

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.10
